In [1]:
import scipy as sci
import numpy as np
import math

In [2]:
c = 3e8
f0 = 9.875e9
lamda = c/f0
H = 500e3
Yc = 750e3
R0 = np.sqrt(Yc**2+H**2)
'''
c:light velocity
f0:radar frequency
lamda:wave length
H:hight
Yc:midle line of detected area
R0:hypotenuse length
'''

'\nc:light velocity\nf0:radar frequency\nlamda:wave length\nH:hight\nYc:midle line of detected area\nR0:hypotenuse length\n'

In [3]:
print(R0)

901387.8188659974


In [4]:
import numpy as np
def nextpow2(n):
    '''
    求最接近数据长度的2的整数次方
    An integer equal to 2 that is closest to the length of the data
    
    Eg: 
    nextpow2(2) = 1
    nextpow2(2**10+1) = 11
    nextpow2(2**20+1) = 21
    '''
    return np.ceil(np.log2(np.abs(n))).astype('long')

In [5]:
theta = np.arcsin(H/R0)
alpha = 30
Rc = R0/np.cos(alpha)
Xc = R0*np.tan(alpha)
Br = 30e6
Vr = 7100
Tr = 20e-6
Kr = Br/Tr
Fr = 1.2*Br
Ts = 1/Fr

In [10]:
Nk = math.ceil((2*1000/c+Tr)/Ts)
Nf = 2**nextpow2(Nk)
tf_ori =  Ts*np.arange(-Nf/2,Nf/2-1)
tf = 2*Rc/c+tf_ori

In [12]:
La = 6
Ls = R0*lamda/La*np.cos(alpha)
Ta = Ls/Vr
Ka = -2*Vr**2*(np.cos(alpha))**2/(lamda*R0)
Ba = 2*Vr*np.cos(alpha)/La
PRF = math.ceil(1.2*Ba)
PRT = 1/PRF
Ns = 2^nextpow2((200/Vr+Ta)*PRF)
ts = np.arange(-Nf/2,Nf/2-1) * PRT
fdc = 2*Vr*np.sin(alpha)/lamda
fdc_ref=fdc
rho_r=c/2/Br
rho_a=Vr*PRT


In [26]:
X0 = np.array([Xc-100,Xc+100,Xc,Xc-100,Xc+100])
Z0 = np.array([0,0,0,0,0])
Y0 = np.array([Yc+100,Yc+100,Yc,Yc-100,Yc-100])
eta_c=(X0-np.sqrt(H**2+Y0**2)*np.tan(alpha))/Vr
print(eta_c)
NT = np.size(X0,0)
print(NT)

[ 0.06098123  0.08915024  0.         -0.08914716 -0.06097815]
5


In [28]:
Sb = np.zeros(Ns,Nf)
sigma = 1
for ii in range(1,NT):
    R = np.sqrt((Vr*ts-X0[ii])^2+Y0[ii]^2+(Z0[ii]-H)^2)
    tau = 2*R/c
    Dfast = np.ones(Ns,1)*tf - tau.T*np.ones(1, Nf)
    phase = np.pi*Kr*Dfast**2-(2*np.pi*f0*tau.T)*np.ones(1,Nf)
    Sb = Sb+sigma*np.exp(1j*phase)*(np.abs(Dfast)<=Tr/2)*((abs(ts-eta_c(ii)) <=Ta/2).T*np.ones(1,Nf))
import matplotlib.pyplot as plt
plt.imshow(real(Sb),cmap='gray')
plt.colorbar()
    


TypeError: ufunc 'bitwise_xor' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''